# SVM hyperparameter optimization

This tutorial provides an example of a simple closed optimization loop on `sklearn`'s Support Vector Machine.

In [1]:
import numpy as np
from ax import (
    ParameterType,
    RangeParameter,
    SearchSpace,
    SimpleExperiment,
    modelbridge,
    models,
)
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.utils.notebook.plotting import render, init_notebook_plotting

In [2]:
init_notebook_plotting()

[INFO 03-15 16:54:39] ipy_plotting: Injecting Plotly library into cell. Do not overwrite or delete cell.


First, we define a simple ML model and prepare data for it:

In [3]:
from sklearn import svm
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

iris = load_iris()
X = iris.data
y = iris.target
X_2d = X[:, :2]
X_2d = X_2d[y > 0]
y_2d = y[y > 0]
y_2d -= 1
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_2d = scaler.fit_transform(X_2d)

Now we define an evaluation function that takes in a parameterization (set of parameter values) and computes all the metrics needed in optimization. It should output a dictionary of metric names to tuple of mean and standard error.

In [4]:
def svm_evaluation_function(
    parameterization, # dict of parameter names to values of those parameters
    weight=None, # required by the evaluation function signature
):
    # create train + test data
    X_train, X_test, y_train, y_test = train_test_split(X_2d, y_2d)

    # fit model
    clf = svm.SVC(gamma=parameterization["gamma"], C=parameterization["C"])
    clf.fit(X_train, y_train)
    
    
    # diagnostics
    accuracy = clf.score(X_test, y_test)
    return {
        'accuracy': (accuracy, 0.0), 
    }

We also define a search space of two SVM hyperparameters, `C` and `gamma`:

In [5]:
svm_search_space = SearchSpace(parameters=[
    RangeParameter(
        name='C', parameter_type=ParameterType.FLOAT, lower=0.1, upper=100, log_scale=True
    ),
    RangeParameter(
        name='gamma', parameter_type=ParameterType.FLOAT, lower=0.001, upper=100, log_scale=True
    ),
])

Finally, we set up a `SimpleExperiment` with our search space and evaluation function. Note that `SimpleExperiment` can be used here instead of `Experiment` because points tried in optimization are computed synchrously via the evaluation function.

In [6]:
exp = SimpleExperiment(
    name='test_svm',
    search_space=svm_search_space,
    evaluation_function=svm_evaluation_function,
    objective_name='accuracy',
)

The resulting hyperparameter optimization loop:

In [7]:
# We only instantiate the Sobol generator once, as the underlying model does not to be re-fit every 
# time new data is added to the experiment.
sobol = modelbridge.get_sobol(search_space=exp.search_space)
print(f"Running Sobol initialization trials...")
for _ in range(5):
    exp.new_trial(generator_run=sobol.gen(1))
for i in range(15):
    print(f"Running GP+EI optimization trial {i+1}/15...")
    # Since we need to re-fit the underlying GP model, we reinstantiate the GP+EI model every 
    # time new data is added to the experiment.
    gpei = modelbridge.get_GPEI(experiment=exp, data=exp.eval())
    exp.new_trial(generator_run=gpei.gen(1))

Running Sobol initialization trials...
Running GP+EI optimization trial 1/15...
Running GP+EI optimization trial 2/15...
Running GP+EI optimization trial 3/15...
Running GP+EI optimization trial 4/15...
Running GP+EI optimization trial 5/15...
Running GP+EI optimization trial 6/15...
Running GP+EI optimization trial 7/15...
Running GP+EI optimization trial 8/15...
Running GP+EI optimization trial 9/15...
Running GP+EI optimization trial 10/15...
Running GP+EI optimization trial 11/15...
Running GP+EI optimization trial 12/15...
Running GP+EI optimization trial 13/15...
Running GP+EI optimization trial 14/15...
Running GP+EI optimization trial 15/15...


We can inspect the evaluation data we observed and plot it:

In [8]:
dat = exp.eval()
dat.df

,arm_name,mean,metric_name,sem,trial_index
0,0_0,0.64,accuracy,0.0,0
1,1_0,0.64,accuracy,0.0,1
2,2_0,0.44,accuracy,0.0,2
3,3_0,0.64,accuracy,0.0,3
4,4_0,0.76,accuracy,0.0,4
5,5_0,0.72,accuracy,0.0,5
6,6_0,0.76,accuracy,0.0,6
7,7_0,0.52,accuracy,0.0,7
8,8_0,0.64,accuracy,0.0,8
9,9_0,0.72,accuracy,0.0,9


In [9]:
render(plot_contour(model=gpei, param_x='C', param_y='gamma', metric_name='accuracy', relative=False))

In [10]:
# `plot_single_method` expects a 2-d array of means, because it expects to average means from multiple 
# optimization runs, so we wrap out best objectives array in another array.
best_objectives = np.array([[trial.objective_mean for trial in exp.trials.values()]])
best_objective_plot = optimization_trace_single_method(
        y=np.maximum.accumulate(best_objectives, axis=1),
)
render(best_objective_plot)